# Read in Poore and Nemecek 2018 data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr

The FixOurFood Dashboard utilizes FAO food supply data to estimate CO2e emissions from consumed food items in the UK. Estimations of emissions per unit mass of consumed food come from Life Cycle Assestment studies (LCA) which aggregate the emissions associated to all the steps of production, management and consumption.

Poore And Nemecek (2018) [1] present a comprehensive list of LCA estimated CO2e emissions for a range of food items, wichi we match here to FAO food items.
Data is open access and can be downloaded from here:
https://www.science.org/doi/suppl/10.1126/science.aaq0216/suppl_file/aaq0216_datas2.xls

This notebook will read the values from this file and generate an xarray dataset. `AgriFoodPy` provides functions to match the listed items here to other item definitions (e.g. FAOSTAT)

In [2]:
datafile = "../../data/food/aaq0216_datas2.xls"

In [3]:
data = pd.read_excel(datafile, sheet_name='Results - Retail Weight', header=(1,2), nrows=43)
# data = data.rename(columns={"Mean.1":"GHGE intensity global mean (kg CO2e/kg)"})
# data[['Product','GHGE intensity global mean (kg CO2e/kg)']]
data

Unnamed: 0_level_0 Land Use (m2/FU)                            \
                     Product         5th pctl 10th pctl    Mean  Median   
0        Wheat & Rye (Bread)             0.98      1.11    3.85    2.70   
1               Maize (Meal)             0.99      1.14    2.94    1.84   
2              Barley (Beer)             0.21      0.26    1.11    0.88   
3                    Oatmeal             2.64      2.85    7.60    7.72   
4                       Rice             0.99      1.10    2.80    2.15   
5                   Potatoes             0.37      0.44    0.88    0.82   
6                    Cassava             0.73      0.76    1.81    1.32   
7                 Cane Sugar             1.14      1.17    2.04    1.78   
8                 Beet Sugar             1.11      1.19    1.83    1.52   
9               Other Pulses             4.08      9.93   15.57   12.24   
10                      Peas             2.28      2.77    7.46    6.73   
11                      Nuts             4.15      4.49   12.96    8.73   
12                Groundnuts             4.22      4.67    9.11    7.87   
13                   Soymilk             0.30      0.34    0.66    0.64   
14                      Tofu             1.57      1.77    3.52    3.41   
15               Soybean Oil             4.77      5.25   10.52    9.61   
16                  Palm Oil             1.37      1.67    2.42    2.39   
17             Sunflower Oil             7.50      8.37   17.66   16.30   
18              Rapeseed Oil             5.01      5.22   10.63    9.42   
19                 Olive Oil             7.85      7.85   26.31   17.29   
20                  Tomatoes             0.07      0.09    0.80    0.17   
21            Onions & Leeks             0.10      0.10    0.39    0.30   
22           Root Vegetables             0.16      0.17    0.33    0.27   
23                 Brassicas             0.19      0.19    0.55    0.34   
24          Other Vegetables             0.17      0.17    0.38    0.19   
25              Citrus Fruit             0.29      0.35    0.86    0.68   
26                   Bananas             0.22      0.28    1.93    1.39   
27                    Apples             0.30      0.31    0.63    0.51   
28          Berries & Grapes             0.23      0.25    2.41    2.58   
29                      Wine             0.86      0.86    1.78    1.64   
30               Other Fruit             0.21      0.21    0.89    0.92   
31                    Coffee             8.30      8.44   21.62   11.90   
32            Dark Chocolate            21.57     34.70   68.96   53.83   
33   Bovine Meat (beef herd)            70.41     82.84  326.21  170.37   
34  Bovine Meat (dairy herd)            12.27     14.39   43.24   25.94   
35             Lamb & Mutton            47.85     60.06  369.81  127.41   
36                  Pig Meat             7.39      7.76   17.36   13.44   
37              Poultry Meat             6.46      6.65   12.22   11.01   
38                      Milk             0.80      1.11    8.95    2.10   
39                    Cheese             7.86      9.55   87.79   20.18   
40                      Eggs             4.25      4.38    6.27    5.71   
41             Fish (farmed)             0.30      0.82    8.41    5.60   
42      Crustaceans (farmed)             0.60      0.61    2.97    0.82   

                        \
   90th pctl 95th pctl   
0       7.87      9.96   
1       5.70      9.01   
2       2.37      2.87   
3      12.88     13.98   
4       6.21      7.21   
5       1.40      1.66   
6       3.19      3.28   
7       3.11      3.53   
8       3.09      3.29   
9      41.25     41.87   
10     14.19     20.47   
11     26.59     26.59   
12     15.38     15.38   
13      0.92      1.07   
14      4.94      5.87   
15     14.64     17.47   
16      2.99      3.29   
17     27.04     29.69   
18     19.04     20.97   
19     36.32     36.32   
20      0.93      5.62   
21      0.62      0.62   
22      0.

In [4]:
impact_names = ["Land Use",
                "GHG Emissions",
                "Acidifying Emissions (IPCC 2013)", 
                "Acidifying Emissions (IPCC 2007)", 
                "Eutrophying Emissions", 
                "Freshwater Withdrawals", 
                "Stress-Weighted Water Use"]

item_names = pd.unique(data["Unnamed: 0_level_0", "Product"])
print(item_names)

['Wheat & Rye (Bread)' 'Maize (Meal)' 'Barley (Beer)' 'Oatmeal' 'Rice'
 'Potatoes' 'Cassava' 'Cane Sugar' 'Beet Sugar' 'Other Pulses' 'Peas'
 'Nuts' 'Groundnuts' 'Soymilk' 'Tofu' 'Soybean Oil' 'Palm Oil'
 'Sunflower Oil' 'Rapeseed Oil' 'Olive Oil' 'Tomatoes' 'Onions & Leeks'
 'Root Vegetables' 'Brassicas' 'Other Vegetables' 'Citrus Fruit' 'Bananas'
 'Apples' 'Berries & Grapes' 'Wine' 'Other Fruit' 'Coffee'
 'Dark Chocolate' 'Bovine Meat (beef herd)' 'Bovine Meat (dairy herd)'
 'Lamb & Mutton' 'Pig Meat' 'Poultry Meat' 'Milk' 'Cheese' 'Eggs'
 'Fish (farmed)' 'Crustaceans (farmed)']


In [5]:
data_means = data.loc[:, (slice(None), "Mean")]
data_means

,Land Use (m2/FU),"GHG Emissions (kg CO2eq/FU, IPCC 2013 incl. CC feedbacks)","GHG Emissions (kg CO2eq/FU, IPCC 2007)","Acidifying Emissions (g SO2eq/FU, CML2 Baseline)","Eutrophying Emissions (g PO43-eq/FU, CML2 Baseline)",Freshwater Withdrawals (L/FU),Stress-Weighted Water Use (L/FU)
,Mean,Mean,Mean,Mean,Mean,Mean,Mean
0,3.85,1.57,1.58,13.35,7.16,647.5,33385.6
1,2.94,1.70,1.68,11.68,4.03,215.7,10863.3
2,1.11,1.18,1.18,6.59,2.33,17.1,696.4
3,7.60,2.48,2.47,10.68,11.23,482.4,18786.2
4,2.80,4.45,3.81,27.19,35.07,2248.4,49576.3
5,0.88,0.46,0.45,3.87,3.48,59.1,2754.2
6,1.81,1.32,1.30,3.42,0.69,0.0,0.0
7,2.04,3.20,3.16,18.02,16.92,620.1,16438.6
8,1.83,1.81,1.80,12.62,5.41,217.7,9493.3


In [6]:
values = data_means.to_numpy()
print(values.shape)

(43, 7)


In [7]:
data = xr.Dataset(
    data_vars = {
        "Land Use":(["Item"], values[:,0]),
        "GHG Emissions":(["Item"], values[:,1]),
        "Acidifying Emissions":(["Item"], values[:,2]),
        "Eutrophying Emissions":(["Item"], values[:,4]),
        "Water Use":(["Item"], values[:,5])
    },
    
    coords = dict(
        Item=("Item", item_names),        
    )
)
data

<xarray.Dataset>
Dimensions:                (Item: 43)
Coordinates:
  * Item                   (Item) object 'Wheat & Rye (Bread)' ... 'Crustacea...
Data variables:
    Land Use               (Item) float64 3.85 2.94 1.11 7.6 ... 6.27 8.41 2.97
    GHG Emissions          (Item) float64 1.57 1.7 1.18 ... 4.67 13.63 26.87
    Acidifying Emissions   (Item) float64 1.58 1.68 1.18 ... 4.6 12.51 23.99
    Eutrophying Emissions  (Item) float64 7.16 4.03 2.33 ... 21.76 235.1 227.2
    Water Use              (Item) float64 647.5 215.7 ... 3.691e+03 3.515e+03

In [8]:
data["Land Use"].sel(Item="Tomatoes")

<xarray.DataArray 'Land Use' ()>
array(0.8)
Coordinates:
    Item     <U8 'Tomatoes'

In [9]:
encoding = {"Land Use":{"dtype":"float32", "zlib": True, "complevel": 5},
           "GHG Emissions":{"dtype":"float32", "zlib": True, "complevel": 5},
           "Acidifying Emissions":{"dtype":"float32", "zlib": True, "complevel": 5},
           "Eutrophying Emissions":{"dtype":"float32", "zlib": True, "complevel": 5},
           "Water Use":{"dtype":"float32", "zlib": True, "complevel": 5}}
data.to_netcdf("../../agrifoodpy/impact/data/PN18.nc", encoding = encoding)